In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from PIL import Image, ImageDraw
import PIL
import torch
import os
import torchvision.transforms.functional as F
import numpy as np
import random
from IPython.display import display
import os
import cv2

# Read Yolo Labels from .txt file:

In [ ]:
def parse_yolo_label(label_str):
  class_label, x_center, y_center, width, height = map(float, label_str.split())
  bbox= [ x_center, y_center, width, height]
  return [int(class_label)  ,bbox]

# Convert normalized Yolo bounding box coordinates to pixel coordinates.


In [ ]:
def xywhn2xyxy(x, w=640, h=640, padw=0, padh=0):
    """
    Convert normalized bounding box coordinates to pixel coordinates.

    Args:
        x (np.ndarray | torch.Tensor): The bounding box coordinates.
        w (int): Width of the image. Defaults to 640
        h (int): Height of the image. Defaults to 640
        padw (int): Padding width. Defaults to 0
        padh (int): Padding height. Defaults to 0
    Returns:
        y (np.ndarray | torch.Tensor): The coordinates of the bounding box in the format [x1, y1, x2, y2] where
            x1,y1 is the top-left corner, x2,y2 is the bottom-right corner of the bounding box.
    """
    assert x.shape[-1] == 4, f"input shape last dimension expected 4 but input shape is {x.shape}"
    y = torch.empty_like(x) if isinstance(x, torch.Tensor) else np.empty_like(x)
    y[..., 0] = w * (x[..., 0] - x[..., 2] / 2) + padw  # top left x
    y[..., 1] = h * (x[..., 1] - x[..., 3] / 2) + padh  # top left y
    y[..., 2] = w * (x[..., 0] + x[..., 2] / 2) + padw  # bottom right x
    y[..., 3] = h * (x[..., 1] + x[..., 3] / 2) + padh  # bottom right y
    return y

# Convert pixel coordinates to normalized Yolo bounding box coordinates:


In [ ]:
def xyxy2xywhn(x, w=640, h=640, clip=False, eps=0.0):
    """
    Convert bounding box coordinates from (x1, y1, x2, y2) format to (x, y, width, height, normalized) format. x, y,
    width and height are normalized to image dimensions.

    Args:
        x (np.ndarray | torch.Tensor): The input bounding box coordinates in (x1, y1, x2, y2) format.
        w (int): The width of the image. Defaults to 640
        h (int): The height of the image. Defaults to 640
        clip (bool): If True, the boxes will be clipped to the image boundaries. Defaults to False
        eps (float): The minimum value of the box's width and height. Defaults to 0.0

    Returns:
        y (np.ndarray | torch.Tensor): The bounding box coordinates in (x, y, width, height, normalized) format
    """
    if clip:
        x = clip_boxes(x, (h - eps, w - eps))
    assert x.shape[-1] == 4, f"input shape last dimension expected 4 but input shape is {x.shape}"
    y = torch.empty_like(x) if isinstance(x, torch.Tensor) else np.empty_like(x)
    y[..., 0] = ((x[..., 0] + x[..., 2]) / 2) / w  # x center
    y[..., 1] = ((x[..., 1] + x[..., 3]) / 2) / h  # y center
    y[..., 2] = (x[..., 2] - x[..., 0]) / w  # width
    y[..., 3] = (x[..., 3] - x[..., 1]) / h  # height
    return y

# Doing Horizontal Flip:

In [ ]:
def horizontal_flip(image, boxes):
    '''
        Flip image horizontally.
        image: a PIL image
        boxes: Bounding boxes, a tensor of dimensions (#objects, 4)
    '''
    new_image = F.hflip(image)

    #flip boxes
    new_boxes = boxes.clone()
    new_boxes[:, 0] = image.width - boxes[:, 0]
    new_boxes[:, 2] = image.width - boxes[:, 2]
    new_boxes = new_boxes[:, [2, 1, 0, 3]]
    return new_image, new_boxes

# Doing Vertical Flip:

In [ ]:
def vertical_flip(image, boxes):
    '''
        Flip image vertically.
        image: a PIL image
        boxes: Bounding boxes, a tensor of dimensions (#objects, 4)
    '''
    new_image = F.vflip(image)

    # Flip boxes
    new_boxes = boxes.clone()
    new_boxes[:, 1] = image.height - boxes[:, 1]
    new_boxes[:, 3] = image.height - boxes[:, 3]
    new_boxes = new_boxes[:, [0, 3, 2, 1]]
    return new_image, new_boxes

# Doing Rotation:

In [ ]:
def rotate(image, boxes, angle,width_image,height_image):
    '''
        Rotate image and bounding box
        image: A Pil image (w, h)
        boxes: A tensors of dimensions (#objects, 4)

        Out: rotated image (w, h), rotated boxes
    '''
    new_image = image.copy()
    new_boxes = boxes.clone()

    w = image.width
    h = image.height
    cx = w/2
    cy = h/2
    new_image = new_image.rotate(angle, expand= True)
    angle = np.radians(angle)
    alpha = np.cos(angle)
    beta = np.sin(angle)
    #Get affine matrix
    AffineMatrix = torch.tensor([[alpha, beta, (1-alpha)*cx - beta*cy],
                                 [-beta, alpha, beta*cx + (1-alpha)*cy]])

    #Rotation boxes
    box_width = (boxes[:,2] - boxes[:,0]).reshape(-1,1)
    box_height = (boxes[:,3] - boxes[:,1]).reshape(-1,1)

    #Get corners for boxes
    x1 = boxes[:,0].reshape(-1,1)
    y1 = boxes[:,1].reshape(-1,1)

    x2 = x1 + box_width
    y2 = y1

    x3 = x1
    y3 = y1 + box_height

    x4 = boxes[:,2].reshape(-1,1)
    y4 = boxes[:,3].reshape(-1,1)

    corners = torch.stack((x1,y1,x2,y2,x3,y3,x4,y4), dim= 1)
    corners = corners.reshape(-1,2)
    corners = torch.cat((corners, torch.ones(corners.shape[0], 1)), dim= 1)

    cos = np.abs(AffineMatrix[0, 0])
    sin = np.abs(AffineMatrix[0, 1])

    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    AffineMatrix[0, 2] += (nW / 2) - cx
    AffineMatrix[1, 2] += (nH / 2) - cy

    #Apply affine transform
    AffineMatrix = AffineMatrix.float()  #
    rotate_corners = torch.mm(AffineMatrix, corners.t()).t()
    rotate_corners = rotate_corners.reshape(-1,8)

    x_corners = rotate_corners[:,[0,2,4,6]]
    y_corners = rotate_corners[:,[1,3,5,7]]

    #Get (x_min, y_min, x_max, y_max)
    x_min, _ = torch.min(x_corners, dim= 1)
    x_min = x_min.reshape(-1, 1)
    y_min, _ = torch.min(y_corners, dim= 1)
    y_min = y_min.reshape(-1, 1)
    x_max, _ = torch.max(x_corners, dim= 1)
    x_max = x_max.reshape(-1, 1)
    y_max, _ = torch.max(y_corners, dim= 1)
    y_max = y_max.reshape(-1, 1)

    new_boxes = torch.cat((x_min, y_min, x_max, y_max), dim= 1)

    scale_x = new_image.width / w
    scale_y = new_image.height / h

    #Resize new image to (w, h)
    new_image = new_image.resize((width_image, height_image))

    #Resize boxes
    new_boxes /= torch.Tensor([scale_x, scale_y, scale_x, scale_y])
    new_boxes[:, 0] = torch.clamp(new_boxes[:, 0], 0, w)
    new_boxes[:, 1] = torch.clamp(new_boxes[:, 1], 0, h)
    new_boxes[:, 2] = torch.clamp(new_boxes[:, 2], 0, w)
    new_boxes[:, 3] = torch.clamp(new_boxes[:, 3], 0, h)
    return new_image, new_boxes

# Start The Augmentaion Process:

# ================================================================================

## Horizontal Flip Part:

In [ ]:
def augment_data(image_path, label_path, save_dir):
  # Read image and label
  image = Image.open(image_path, mode= "r")
  image = image.convert("RGB")
  width, height = image.size
  with open(label_path, 'r') as f:
    labels = [parse_yolo_label(line.strip()) for line in f.readlines()]
  bbox = [label[1] for label in labels]
  class_label = [label[0] for label in labels]
  flat_list = [item for sublist in bbox for item in sublist]
  xy_list = flat_list
  xy_tensor = torch.tensor(xy_list).reshape(-1, 4)
  xy_coords = xywhn2xyxy(xy_tensor, width, height)


  boxes = torch.FloatTensor(xy_coords)
  torch.set_printoptions(sci_mode=False, precision=15)

  # Apply Horizontal Flip for image and boxes
  image_Hflip, new_boxes = horizontal_flip(image, boxes)


  yolo_list = new_boxes
  yolo_tensor = torch.tensor(yolo_list)
  yolo_coords = xyxy2xywhn(yolo_tensor, width, height)


  coords_list = yolo_coords.tolist()

  i=0
   # Generate unique filenames
  filename, ext = os.path.splitext(os.path.basename(image_path))
  new_filename = f"{filename}_aug_{i+1}HF{ext}"
  save_path = os.path.join(save_dir, new_filename)
  image_Hflip.save(save_path)

  with open(os.path.join(save_dir, f"{new_filename[:-len(ext)]}.txt"), 'w') as f:
        for label, coord in zip(class_label, coords_list):
            # Write augmented class ID
            f.write(f"{label} ")

            # Write bounding box coordinates from augmented_labels
            f.write(' '.join(map(str, coord)) + '\n')

  print("Image ", new_filename , "Done Successfuly")
  print("======================================================================")



# Modify these paths according to your dataset
image_dir = ""
label_dir = ""
save_dir = ""


count = 0
# Loop through images and labels
for image_filename in os.listdir(image_dir):
  image_path = os.path.join(image_dir, image_filename)
  label_path = os.path.join(label_dir, os.path.splitext(image_filename)[0] + ".txt")
  augment_data(image_path, label_path, save_dir)
  count += 1

print("Number of Images Done: ", count)
print("Data augmentation completed!")

<ipython-input-8-ae22cce6ca83>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yolo_tensor = torch.tensor(yolo_list)


Streaming output truncated to the last 5000 lines.
Image  p (477)_aug_1HF.jpg Done Successfuly
Image  p (397)_aug_1HF.jpg Done Successfuly
Image  p (403)_aug_1HF.jpg Done Successfuly
Image  p (419)_aug_1HF.jpg Done Successfuly
Image  p (467)_aug_1HF.jpg Done Successfuly
Image  p (409)_aug_1HF.jpg Done Successfuly
Image  p (474)_aug_1HF.jpg Done Successfuly
Image  p (471)_aug_1HF.jpg Done Successfuly
Image  p (44)_aug_1HF.jpg Done Successfuly
Image  p (449)_aug_1HF.jpg Done Successfuly
Image  p (404)_aug_1HF.jpg Done Successfuly
Image  p (423)_aug_1HF.jpg Done Successfuly
Image  p (459)_aug_1HF.jpg Done Successfuly
Image  p (435)_aug_1HF.jpg Done Successfuly
Image  p (401)_aug_1HF.jpg Done Successfuly
Image  p (466)_aug_1HF.jpg Done Successfuly
Image  p (46)_aug_1HF.jpg Done Successfuly
Image  p (468)_aug_1HF.jpg Done Successfuly
Image  p (427)_aug_1HF.jpg Done Successfuly
Image  p (442)_aug_1HF.jpg Done Successfuly
Image  p (454)_aug_1HF.jpg Done Successfuly
Image  p (443)_aug_1HF.jpg 

## Vertical Flip Part:

In [ ]:
def augment_data(image_path, label_path, save_dir):
  image = Image.open(image_path, mode= "r")
  image = image.convert("RGB")
  width, height = image.size
  with open(label_path, 'r') as f:
    labels = [parse_yolo_label(line.strip()) for line in f.readlines()]

  bbox = [label[1] for label in labels]
  class_label = [label[0] for label in labels]

  flat_list = [item for sublist in bbox for item in sublist]
  xy_list = flat_list
  xy_tensor = torch.tensor(xy_list).reshape(-1, 4)
  xy_coords = xywhn2xyxy(xy_tensor, width, height)


  boxes = torch.FloatTensor(xy_coords)
  torch.set_printoptions(sci_mode=False, precision=15)

  # Apply Vertical Flip for image and boxes
  image_Vflip, new_boxes = vertical_flip(image, boxes)

  yolo_list = new_boxes
  yolo_tensor = torch.tensor(yolo_list)
  yolo_coords = xyxy2xywhn(yolo_tensor, width, height)


  coords_list = yolo_coords.tolist()

  i=0
   # Generate unique filenames
  filename, ext = os.path.splitext(os.path.basename(image_path))
  new_filename = f"{filename}_aug_{i+1}VF{ext}"
  save_path = os.path.join(save_dir, new_filename)
  image_Vflip.save(save_path)

  with open(os.path.join(save_dir, f"{new_filename[:-len(ext)]}.txt"), 'w') as f:
        for label, coord in zip(class_label, coords_list):
            # Write augmented class ID
            f.write(f"{label} ")

            # Write bounding box coordinates from augmented_labels
            f.write(' '.join(map(str, coord)) + '\n')

  print("Image ", new_filename , "Done Successfuly")
  print("======================================================================")





# Modify these paths according to your dataset
image_dir = ""
label_dir = ""
save_dir = ""


count = 0
# Loop through images and labels
for image_filename in os.listdir(image_dir):
  image_path = os.path.join(image_dir, image_filename)
  label_path = os.path.join(label_dir, os.path.splitext(image_filename)[0] + ".txt")
  augment_data(image_path, label_path, save_dir)
  count += 1
  print(count)

print("Number of Images Done: ", count)
print("Data augmentation completed!")

<ipython-input-9-ebbf8950ccf9>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yolo_tensor = torch.tensor(yolo_list)


Streaming output truncated to the last 5000 lines.
Image  Tr-gl_0163_aug_1VF.jpg Done Successfuly
3072
Image  Tr-gl_0155_aug_1VF.jpg Done Successfuly
3073
Image  Tr-gl_0154_aug_1VF.jpg Done Successfuly
3074
Image  Tr-gl_0170_aug_1VF.jpg Done Successfuly
3075
Image  Tr-gl_0160_aug_1VF.jpg Done Successfuly
3076
Image  Tr-gl_0136_aug_1VF.jpg Done Successfuly
3077
Image  Tr-gl_0164_aug_1VF.jpg Done Successfuly
3078
Image  Tr-gl_0138_aug_1VF.jpg Done Successfuly
3079
Image  Tr-gl_0176_aug_1VF.jpg Done Successfuly
3080
Image  Tr-gl_0174_aug_1VF.jpg Done Successfuly
3081
Image  Tr-gl_0177_aug_1VF.jpg Done Successfuly
3082
Image  Tr-gl_0165_aug_1VF.jpg Done Successfuly
3083
Image  Tr-gl_0169_aug_1VF.jpg Done Successfuly
3084
Image  Tr-gl_0153_aug_1VF.jpg Done Successfuly
3085
Image  Tr-gl_0173_aug_1VF.jpg Done Successfuly
3086
Image  Tr-gl_0175_aug_1VF.jpg Done Successfuly
3087
Image  Tr-gl_0178_aug_1VF.jpg Done Successfuly
3088
Image  Tr-gl_0180_aug_1VF.jpg Done Successfuly
3089
Image  Tr-gl_

## Rotation by 90 degree Part:

In [ ]:
def augment_data(image_path, label_path, save_dir):
  image = Image.open(image_path, mode= "r")
  image = image.convert("RGB")
  width, height = image.size
  with open(label_path, 'r') as f:
    labels = [parse_yolo_label(line.strip()) for line in f.readlines()]

  bbox = [label[1] for label in labels]
  class_label = [label[0] for label in labels]

  flat_list = [item for sublist in bbox for item in sublist]
  xy_list = flat_list
  xy_tensor = torch.tensor(xy_list).reshape(-1, 4)
  xy_coords = xywhn2xyxy(xy_tensor, width, height)


  boxes = torch.FloatTensor(xy_coords)
  torch.set_printoptions(sci_mode=False, precision=15)

  # Apply Rotation by 90 for image and boxes
  image_rotate, new_boxes = rotate(image, boxes,90,width,height)


  yolo_list = new_boxes
  yolo_tensor = torch.tensor(yolo_list)
  yolo_coords = xyxy2xywhn(yolo_tensor, width, height)


  coords_list = yolo_coords.tolist()

  i=0
   # Generate unique filenames
  filename, ext = os.path.splitext(os.path.basename(image_path))
  new_filename = f"{filename}_aug_{i+1}R90{ext}"
  save_path = os.path.join(save_dir, new_filename)
  image_rotate.save(save_path)

  with open(os.path.join(save_dir, f"{new_filename[:-len(ext)]}.txt"), 'w') as f:
        for label, coord in zip(class_label, coords_list):
            # Write augmented class ID
            f.write(f"{label} ")

            # Write bounding box coordinates from augmented_labels
            f.write(' '.join(map(str, coord)) + '\n')

  print("Image ", new_filename , "Done Successfuly")
  print("======================================================================")



# Modify these paths according to your dataset
image_dir = ""
label_dir = ""
save_dir = ""


count = 0
# Loop through images and labels
for image_filename in os.listdir(image_dir):
  image_path = os.path.join(image_dir, image_filename)
  label_path = os.path.join(label_dir, os.path.splitext(image_filename)[0] + ".txt")
  augment_data(image_path, label_path, save_dir)
  count += 1
  print(count)

print("Number of Images Done: ", count)
print("Data augmentation completed!")

<ipython-input-9-f0bda74a42bb>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yolo_tensor = torch.tensor(yolo_list)


Streaming output truncated to the last 5000 lines.
Image  Tr-gl_0163_aug_1R90.jpg Done Successfuly
3072
Image  Tr-gl_0155_aug_1R90.jpg Done Successfuly
3073
Image  Tr-gl_0154_aug_1R90.jpg Done Successfuly
3074
Image  Tr-gl_0170_aug_1R90.jpg Done Successfuly
3075
Image  Tr-gl_0160_aug_1R90.jpg Done Successfuly
3076
Image  Tr-gl_0136_aug_1R90.jpg Done Successfuly
3077
Image  Tr-gl_0164_aug_1R90.jpg Done Successfuly
3078
Image  Tr-gl_0138_aug_1R90.jpg Done Successfuly
3079
Image  Tr-gl_0176_aug_1R90.jpg Done Successfuly
3080
Image  Tr-gl_0174_aug_1R90.jpg Done Successfuly
3081
Image  Tr-gl_0177_aug_1R90.jpg Done Successfuly
3082
Image  Tr-gl_0165_aug_1R90.jpg Done Successfuly
3083
Image  Tr-gl_0169_aug_1R90.jpg Done Successfuly
3084
Image  Tr-gl_0153_aug_1R90.jpg Done Successfuly
3085
Image  Tr-gl_0173_aug_1R90.jpg Done Successfuly
3086
Image  Tr-gl_0175_aug_1R90.jpg Done Successfuly
3087
Image  Tr-gl_0178_aug_1R90.jpg Done Successfuly
3088
Image  Tr-gl_0180_aug_1R90.jpg Done Successfuly


## Rotation by 270 degree part:

In [ ]:
def augment_data(image_path, label_path, save_dir):
  image = Image.open(image_path, mode= "r")
  image = image.convert("RGB")
  width, height = image.size
  with open(label_path, 'r') as f:
    labels = [parse_yolo_label(line.strip()) for line in f.readlines()]

  bbox = [label[1] for label in labels]
  class_label = [label[0] for label in labels]

  flat_list = [item for sublist in bbox for item in sublist]
  xy_list = flat_list
  xy_tensor = torch.tensor(xy_list).reshape(-1, 4)
  xy_coords = xywhn2xyxy(xy_tensor, width, height)


  boxes = torch.FloatTensor(xy_coords)
  torch.set_printoptions(sci_mode=False, precision=15)

 # Apply Rotation by 270 for image and boxes
  image_rotate, new_boxes = rotate(image, boxes,270,width,height)



  yolo_list = new_boxes
  yolo_tensor = torch.tensor(yolo_list)
  yolo_coords = xyxy2xywhn(yolo_tensor, width, height)


  coords_list = yolo_coords.tolist()

  i=0
   # Generate unique filenames
  filename, ext = os.path.splitext(os.path.basename(image_path))
  new_filename = f"{filename}_aug_{i+1}R270{ext}"
  save_path = os.path.join(save_dir, new_filename)
  image_rotate.save(save_path)

  with open(os.path.join(save_dir, f"{new_filename[:-len(ext)]}.txt"), 'w') as f:
        for label, coord in zip(class_label, coords_list):
            # Write augmented class ID
            f.write(f"{label} ")

            # Write bounding box coordinates from augmented_labels
            f.write(' '.join(map(str, coord)) + '\n')

  print("Image ", new_filename , "Done Successfuly")
  print("======================================================================")




# Modify these paths according to your dataset
image_dir = ""
label_dir = ""
save_dir = ""


count = 0
# Loop through images and labels
for image_filename in os.listdir(image_dir):
  image_path = os.path.join(image_dir, image_filename)
  label_path = os.path.join(label_dir, os.path.splitext(image_filename)[0] + ".txt")
  augment_data(image_path, label_path, save_dir)
  count += 1
  print(count)

print("Number of Images Done: ", count)
print("Data augmentation completed!")

<ipython-input-9-33318d4642ee>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yolo_tensor = torch.tensor(yolo_list)


Streaming output truncated to the last 5000 lines.
Image  Tr-gl_0163_aug_1R270.jpg Done Successfuly
3072
Image  Tr-gl_0155_aug_1R270.jpg Done Successfuly
3073
Image  Tr-gl_0154_aug_1R270.jpg Done Successfuly
3074
Image  Tr-gl_0170_aug_1R270.jpg Done Successfuly
3075
Image  Tr-gl_0160_aug_1R270.jpg Done Successfuly
3076
Image  Tr-gl_0136_aug_1R270.jpg Done Successfuly
3077
Image  Tr-gl_0164_aug_1R270.jpg Done Successfuly
3078
Image  Tr-gl_0138_aug_1R270.jpg Done Successfuly
3079
Image  Tr-gl_0176_aug_1R270.jpg Done Successfuly
3080
Image  Tr-gl_0174_aug_1R270.jpg Done Successfuly
3081
Image  Tr-gl_0177_aug_1R270.jpg Done Successfuly
3082
Image  Tr-gl_0165_aug_1R270.jpg Done Successfuly
3083
Image  Tr-gl_0169_aug_1R270.jpg Done Successfuly
3084
Image  Tr-gl_0153_aug_1R270.jpg Done Successfuly
3085
Image  Tr-gl_0173_aug_1R270.jpg Done Successfuly
3086
Image  Tr-gl_0175_aug_1R270.jpg Done Successfuly
3087
Image  Tr-gl_0178_aug_1R270.jpg Done Successfuly
3088
Image  Tr-gl_0180_aug_1R270.jpg